In [69]:
import pandas as pd
import numpy as np
import datetime as dt
from w3lib.html import remove_tags

In [43]:
import os
while not os.getcwd().endswith("Data Cleaning"):
    os.chdir("..")

os.getcwd()

'c:\\Users\\gilnr\\OneDrive - NOVASBE\\Work Project\\Thesis - Code\\Data Cleaning'

In [44]:
from pathlib import Path, PureWindowsPath

main_folder = PureWindowsPath("c:\\Users\\gilnr\\OneDrive - NOVASBE\\Work Project\\Thesis - Code")
MAIN_FOLDER = Path(main_folder)
DATA_FOLDER = MAIN_FOLDER / "Data"
DATA_FOLDER

WindowsPath('c:/Users/gilnr/OneDrive - NOVASBE/Work Project/Thesis - Code/Data')

# Load all datasets 

In [45]:
bons_empregos = pd.read_json(DATA_FOLDER / 'bons_empregos_jobs.json')
career_jet = pd.read_json(DATA_FOLDER / 'career_jet_api.json')
carga_de_trabalhos = pd.read_json(DATA_FOLDER / 'CargaDeTrabalhos.json', lines=True)
emprego_xl = pd.read_json(DATA_FOLDER / 'emprego_xl_jobs.json')
emprego_org = pd.read_json(DATA_FOLDER / 'empregoOrg_jobs.json')
itjobs = pd.read_json(DATA_FOLDER / 'itjobs_api.json')
jooble = pd.read_json(DATA_FOLDER / 'jooble_api.json')
landing_jobs = pd.read_json(DATA_FOLDER / 'landingjobs_api.json')
net_empregos = pd.read_json(DATA_FOLDER / 'net_empregos.json')

# Data Cleaning
- For each website there are specific categories that we need to attend. Some require filtering for job location, others cleaning the job description, and the post date.

## Drop Job Vacancies
What makes a unique job vacancy?
- For our analysis it will be: [job_title, job_description, company, job_location]

# General Functions

In [46]:
def description():
    """Clean job description of:
    Weird characters (\n, \r, \r\n,-, etc...)
    """
    pass

In [135]:
def copy_df(dataframe):
   return dataframe.copy()

def replacenan(dataframe):
    dataframe.replace('nan', np.nan, inplace=True)
    return dataframe
    
def dropNullJobs(dataframe):
    """
    Drop null values that make an online job vacancy unusable for analysis.
    The subset to drop is: ['post_date', 'job_title', 'job_description']
    """
    dataframe.dropna(subset=['post_date', 'job_title', 'job_description'], inplace=True)
    return dataframe

# Convert Scrape date to datetime
def toDatetime(dataFrame, columns_list):
    for i in columns_list:
        dataFrame[i] = pd.to_datetime(dataFrame[i])
    return dataFrame

# remove duplicates
def removeDupes(dataframe, subset=['job_title', 'job_description', 'company', 'job_location']):
    dataframe.sort_values(by='post_date').drop_duplicates(subset=subset, keep='last', inplace=True)
    return dataframe

# convert portuguese months to numbers
def longToShortDate(x, sep):
    months = ['janeiro', 'fevereiro','março', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro']
    months_dic = {value:idx+1 for idx, value in enumerate(months)}
    date = [i.strip() for i in x.split(sep)]
    return f'{date[0]}/{months_dic[date[1]]}/{date[2]}'

# convert to datetime object
def convertToDatetime(dataframe, function, sep=' '):
    # Remove comma from date
    dataframe['post_date'] = dataframe['post_date'].apply(lambda x: str(x).lower().replace(',',''))
    dataframe['post_date'] = dataframe['post_date'].apply(lambda x: dt.datetime.strptime(function(x, sep), "%d/%m/%Y"))
    return dataframe

def listToRows(dataframe, column):
    return dataframe.explode(column)

def removeTags(dataframe, column_list):
    for i in column_list:
        dataframe[i] = dataframe[i].apply(remove_tags)
    return dataframe

def postDatePreprocess(dataframe, sep=" "):
    dataframe['post_date'] = dataframe['post_date'].apply(lambda x: x.split(sep)[0]) 
    return dataframe

# Description
def clean_text(text):
    to_replace = ['\r', '\n', '•']
    replace = ['', '', '\n']

    for idx, val in enumerate(to_replace):
        text = text.replace(val, replace[idx])
    text = text.strip()
    return text

def cleanDescription(dataframe, column_list):
    for i in column_list:
        dataframe[i] = dataframe[i].apply(lambda x: clean_text(x))
    return dataframe

## Bons Empregos
- Specific functions:
    - `getPortugalLocation`

In [48]:
def getPortugalLocation(dataframe):
    # Get only job offers in Portugal
    dataframe = dataframe.loc[dataframe['job_location'] != 'Estrangeiro'].copy()
    return dataframe

In [49]:
bons_empregos_clean = (bons_empregos.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(dropNullJobs).
                    pipe(toDatetime, ['scrape_date']).
                    pipe(getPortugalLocation).
                    pipe(convertToDatetime, longToShortDate).
                    pipe(removeDupes)
)

print(f'Previous shape: {bons_empregos.shape}\nCurrent shape:{bons_empregos_clean.shape}')

Previous shape: (2576, 8)
Current shape:(2357, 8)


## Career Jet

In [50]:
# convert job location to list
career_jet['job_location'] = career_jet['job_location'].apply(lambda x: x.split(','))


In [51]:
career_jet_clean = (career_jet.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(dropNullJobs).
                    pipe(toDatetime, ['scrape_date', 'post_date']).
                    pipe(listToRows, 'job_location').
                    pipe(removeDupes)
)
print(f'Previous shape: {career_jet.shape}\nCurrent shape:{career_jet_clean.shape}')

Previous shape: (6534, 9)
Current shape:(10613, 9)


In [59]:
assert career_jet_clean.post_date.dtypes == career_jet_clean.scrape_date.dtypes

## Carga de Trabalhos

In [60]:
carga_de_trabalhos_clean = (carga_de_trabalhos.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(dropNullJobs).
                    pipe(toDatetime, ['scrape_date']).
                    pipe(convertToDatetime, longToShortDate, '/').
                    pipe(removeDupes)
)

print(f'Previous shape: {carga_de_trabalhos.shape}\nCurrent shape:{carga_de_trabalhos_clean.shape}')

Previous shape: (80, 7)
Current shape:(80, 7)


## Emprego XL

In [62]:
# emprego_xl_clean

## Emprego.Org `SCRAPE AGAIN WITH SCRAPY FOR THE CORRECT FIELDS`

In [ ]:
# emprego_org_clean = (emprego_org.
#                     pipe(copy_df).
#                     pipe(replacenan).
#                     pipe(dropNullJobs).
#                     pipe(postDatePreprocess, '/').
#                     pipe(toDatetime, ['scrape_date', 'post_date']).
#                     pipe(removeDupes)
# )

## ITJOBS

In [67]:
itjobs_clean = (itjobs.
                    pipe(copy_df).
                    pipe(listToRows, 'job_location').
                    pipe(replacenan).
                    pipe(dropNullJobs).
                    pipe(toDatetime, ['scrape_date', 'post_date']).
                    pipe(removeDupes)
)

print(f'Previous shape: {itjobs.shape}\nCurrent shape:{itjobs_clean.shape}')

Previous shape: (3890, 9)
Current shape:(5076, 9)


## Jooble

In [78]:
jooble_clean = (jooble.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(dropNullJobs).
                    pipe(toDatetime, ['scrape_date', 'post_date']).
                    pipe(removeTags, ['job_title']).
                    pipe(removeDupes)
)

print(f'Previous shape: {jooble.shape}\nCurrent shape:{jooble_clean.shape}')

Previous shape: (1093, 9)
Current shape:(1093, 9)


## Landing Jobs IT

In [84]:
landing_jobs_clean = (landing_jobs.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(dropNullJobs).
                    pipe(postDatePreprocess, 'T').
                    pipe(toDatetime, ['scrape_date', 'post_date']).
                    pipe(removeTags, 'job_title').
                    pipe(removeDupes)
)

print(f'Previous shape: {landing_jobs.shape}\nCurrent shape:{landing_jobs_clean.shape}')

Previous shape: (717, 9)
Current shape:(717, 9)


## Net Empregos

In [128]:
net_empregos_clean = (net_empregos.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(dropNullJobs).
                    pipe(cleanDescription, ['job_title']).
                    pipe(toDatetime, ['scrape_date', 'post_date']).
                    pipe(removeDupes)
)

print(f'Previous shape: {net_empregos.shape}\nCurrent shape:{net_empregos_clean.shape}')

Previous shape: (54800, 8)
Current shape:(54791, 8)


# Add Website Column to all dataframes before concat

In [129]:
jobs_dfs = [bons_empregos_clean, career_jet_clean, carga_de_trabalhos_clean, #emprego_xl_clean, emprego_org_clean, 
            itjobs_clean, jooble_clean, landing_jobs_clean, net_empregos_clean]
websites = ['Bons empregos', 'Career Jet', 'Carga de Trabalhos',#'Emprego XL', 'Emprego.org' 
            'ITjobs','Jooble','Landing Jobs','Net-empregos']

# Add column with website name
for idx, value in enumerate(jobs_dfs):
    value['website'] = websites[idx]   

# Concat All dataframes into one for data Deduplication

In [130]:
neworder = ['job_title','job_description','company','job_location','job_category','salary', 'post_date', 'scrape_date','job_href', 'website']

df = pd.concat([i.reindex(columns=neworder) for i in jobs_dfs])

# Validate that the concatenation is happening properly
assert len(df) == sum(len(i) for i in jobs_dfs)

In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74727 entries, 10 to 54799
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   job_title        74727 non-null  object        
 1   job_description  74727 non-null  object        
 2   company          73987 non-null  object        
 3   job_location     74727 non-null  object        
 4   job_category     74647 non-null  object        
 5   salary           12483 non-null  object        
 6   post_date        74727 non-null  datetime64[ns]
 7   scrape_date      74727 non-null  datetime64[ns]
 8   job_href         67841 non-null  object        
 9   website          74727 non-null  object        
dtypes: datetime64[ns](2), object(8)
memory usage: 6.3+ MB


## Pipeline

In [141]:
def cleanCompany(dataframe):
    def capitalize(x):
        try:
            return x.capitalize()
        except AttributeError:
            return ''
    dataframe['company'] = dataframe['company'].apply(lambda x: capitalize(x))
    return dataframe

In [154]:
df_clean = (df.
            pipe(copy_df).
            pipe(replacenan).
            pipe(dropNullJobs).
            pipe(cleanCompany).
            pipe(cleanDescription, ['job_title', 'job_description']).
            pipe(removeDupes, ['job_title', 'company', 'job_location'])
)
df_clean.reset_index(drop=True, inplace=True)

print(f'Previous shape: {df.shape}\nCurrent shape:{df_clean.shape}')

Previous shape: (74727, 10)
Current shape:(74727, 10)


In [155]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74727 entries, 0 to 74726
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   job_title        74727 non-null  object        
 1   job_description  74727 non-null  object        
 2   company          74727 non-null  object        
 3   job_location     74727 non-null  object        
 4   job_category     74647 non-null  object        
 5   salary           12483 non-null  object        
 6   post_date        74727 non-null  datetime64[ns]
 7   scrape_date      74727 non-null  datetime64[ns]
 8   job_href         67841 non-null  object        
 9   website          74727 non-null  object        
dtypes: datetime64[ns](2), object(8)
memory usage: 5.7+ MB


In [156]:
print(f'There is a total of {len(df_clean)} jobs of which {df_clean.job_title.nunique()} have unique titles')

There is a total of 74727 jobs of which 53415 have unique titles


In [157]:
df.describe()

<ipython-input-157-ea8415b8a3ee>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()
<ipython-input-157-ea8415b8a3ee>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()


,job_title,job_description,company,job_location,job_category,salary,post_date,scrape_date,job_href,website
count,74727,74727,73987,74727,74647,12483,74727,74727,67841,74727
unique,53434,53279,12930,538,68,143,4154,3,58281,7
top,Ajudante de Cozinha,\r\n,,Lisboa,,,2021-10-22 00:00:00,2021-05-10 00:00:00,http://jobviewtrack.com/pt-pt/job-1d12416e4c16...,Net-empregos
freq,401,5254,7497,20559,17499,6030,10231,57130,40,54791
first,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-16 00:00:00,2021-04-10 00:00:00,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-09 00:00:00,2021-10-22 00:00:00,NaN,NaN


In [158]:
df_clean.to_json(DATA_FOLDER / 'full_data_clean.json')